In [ ]:
# conexão com o banco de dados do data warehouse e com o Banco de dados original
import psycopg2 as pg
import pyodbc as odbc

# Ferramentas data science
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Variaveis de ambiente E conexão com o Banco

In [ ]:
# engine = pg.connect(
#     "dbname='lippaus' user='postgres' host='192.168.0.7' port='5432' password='#L!pp@us2@18'")

In [ ]:
cnxn = odbc.connect("Driver={DBMaker 5.2 Driver};"
                        "Server=18.228.164.49;"
                        "Database=DBCONTROL_2016_001;"
                        "uid=DBCONTROLTABLEAU;pwd=123456")
cursor = cnxn.cursor()

### Querys

In [ ]:
# df = pd.read_sql('SELECT * FROM "contabil" ', con=engine)

In [ ]:
dfb = pd.read_sql('''
SELECT 

    CASE WHEN 
        CT.CGLANLAN_PLA_CTA >= 3000000000000 THEN 'DRE'
        WHEN CT.CGLANLAN_PLA_CTA <= 2999999999999 THEN 'PAT'
        ELSE 'ERROR'
    END AS "TIPO DE CONTA",
    
    -- DATAS
    LEFT(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),4) AS "ANO",
    SUBSTRING(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),5,2) AS "MES",
    LEFT(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),4) ||
    SUBSTRING(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),5,2) AS "MESANO",
    CT.CGLANLAN_DATA AS "DATA EMISSAO",

    -- DIMENSION
    CT.CGLANLAN_FILIAL AS "COD FILIAL",
    CT.CGLANLAN_NDOC AS "NUMERO DOCUMENTO",
    CT.CGLANLAN_IDLT AS "TIPO PLANO CONTABIL",

    -- CODIGO DA CONTA
    LTRIM(CAST(CT.CGLANLAN_PLA_CTA AS VARCHAR(20))) AS "NUMERO DA CONTA",
    RIGHT(CAST(CT.CGLANLAN_PLA_CTA AS VARCHAR(20)),6) AS "COD CONTA",

    -- VALORES 
    SUM(CT.CGLANLAN_VALOR) AS "VALOR",
    RTRIM(CT.CGLANLAN_HISTORICO) AS "DESCRICAO",

    CASE
        WHEN CT.CGLANLAN_TPLANC = 1 THEN 'DEBITO'
        ELSE 'CREDITO'
    END AS "DEBITO OU CREDITO"


FROM 
    DBCONTROL2016001.CT020122 AS CT
GROUP BY 

    CASE WHEN 
        CT.CGLANLAN_PLA_CTA >= 3000000000000 THEN 'DRE'
        WHEN CT.CGLANLAN_PLA_CTA <= 2999999999999 THEN 'PAT'
        ELSE 'ERROR'
    END,
    
    -- DATAS
    LEFT(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),4) ,
    SUBSTRING(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),5,2),
    LEFT(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),4) ||
    SUBSTRING(CAST(CT.CGLANLAN_DATA AS VARCHAR(8)),5,2),
    CT.CGLANLAN_DATA,

    -- DIMENSION
    CT.CGLANLAN_FILIAL,
    CT.CGLANLAN_NDOC,
    CT.CGLANLAN_IDLT,

    -- CODIGO DA CONTA
    LTRIM(CAST(CT.CGLANLAN_PLA_CTA AS VARCHAR(20))),
    RIGHT(CAST(CT.CGLANLAN_PLA_CTA AS VARCHAR(20)),6),

    -- VALORES 
    RTRIM(CT.CGLANLAN_HISTORICO),

    CASE
        WHEN CT.CGLANLAN_TPLANC = 1 THEN 'DEBITO'
        ELSE 'CREDITO'
    END''', con=cnxn)

### Analises Exploratoria dados

In [ ]:
# Informaçoes sobre os tipos de dados do dataset
dfb.info()

In [ ]:
# Resumo estatístico do DataFrame, com quartis, mediana, etc.
dfb.describe()

In [ ]:
# Inicio do dataframe
dfb.head()

In [ ]:
# Fim do dataframe
dfb.tail()

In [ ]:
# Quantidade de nulos 
dfb.isnull().sum()

In [ ]:
# Nome das colunas
dfb.columns

In [ ]:
# Numero de indexes
dfb.index

In [ ]:
# Contagem de linhas nao nulas no dataset
dfb.count()

In [ ]:
# Numero de linhas e de colunas
dfb.shape

In [ ]:
# Soma dos valores de um DataFrame
dfb['VALOR'].sum()

In [ ]:
# Menor valor de um DataFrame
dfb['VALOR'].min()

In [ ]:
# Maior valor
dfb['VALOR'].max()

In [ ]:
# Index do menor valor
dfb['VALOR'].idxmin()

In [ ]:
# Index do maior valor
dfb['VALOR'].idxmax()

In [ ]:
# Média dos valores
dfb['VALOR'].mean()

In [ ]:
# Mediana dos valores
dfb['VALOR'].median()

In [ ]:
# # Aplicando uma função que substitui a por b
dfb.apply(lambda x: x.replace('a', 'b'))

In [ ]:
# Ordenando em ordem crescente
dfb['VALOR'] = dfb['VALOR'].sort_values()
dfb['VALOR']

In [ ]:
# Ordenando em ordem decrescente
dfb['VALOR'].sort_values(ascending=False)

In [ ]:
# Removendo linhas pelo index
dfb.drop([0, 1])

In [ ]:
# Removendo colunas utilizando o argumento axis=1
dfb.drop('País', axis=1)

In [ ]:
# Criando uma nova coluna em um DataFrame para dados fixos:
dfb['Nova Coluna'] = 0

In [ ]:
# Se seu DataFrame possui 3 colunas, passe 3 novos valores de nomes em uma lista
dfb.columns = ['Coluna 1', 'Coluna 2', 'Coluna 3']

In [ ]:
# Filtrando o DataFrame para mostrar apenas valores pares (e como se fosse um where no sql)
dfb[dfb['VALOR'] % 2 == 0]

In [ ]:
# Selecionando a primeira linha da coluna numero documento
dfb.loc[0, 'NUMERO DOCUMENTO']

In [ ]:
# Operações matemáticas eo outras manipulações com vários campos 

s = pd.Series([1, 2, 3, 4], index=['VALOR ABERTO', 'VALOR', 'VALOR FECHADO', 'VALOR BAIXA '])
# Somando todos os valores presentes na Series por 2
s.add(2)
# Subtraindo 2 de todos os valores
s.sub(2)
# Multiplicando todos os valores por 2
s.mul(2)
# Dividindo valores por 2
s.div(2)

In [ ]:
sns.barplot(x='TIPO DE CONTA', y='VALOR', data=dfb);

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.catplot(x='TIPO DE CONTA', y='VALOR',data=dfb)
plt.show()

In [ ]:
dfb.head()

In [ ]:
# Converter tipo de dado para tirar espaço em branco
dfb['DESCRICAO'] = dfb['DESCRICAO'].str.strip()

In [ ]:
dfb['DATA EMISSAO'] = dfb['DATA EMISSAO'].astype(str)
dfb['COD FILIAL'] = dfb['COD FILIAL'].astype(str)
dfb['NUMERO DOCUMENTO'] = dfb['NUMERO DOCUMENTO'].astype(str)

In [ ]:
dfb.info()

In [ ]:
dfb.head()

### Gerando arquivo para o ETL

In [ ]:
# dfb.to_csv("C:/Users/tasio.guimaraes/Documents/Contabil/Python/Data/2021.csv")